In [8]:
import pprint
import tweepy
import pymongo

In [20]:
client=pymongo.MongoClient()
db=client["Twitter"]
print db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'Twitter')


## Note: My credentials Is stored in mongoDb database

In [23]:
cred=db.login.find_one({})

In [25]:
#Tweepy Login credensials are stored in mongodb databse
auth = tweepy.OAuthHandler(cred["consumerKey"], cred["consumerSecret"])
auth.set_access_token(cred["oauthTocken"], cred["oauthTokenSecret"])
api = tweepy.API(auth)
api

<h2>Where On Earth ID (WOEID)</h2>
Extract WOEID for most common places & store for future reference, Becouse query is limited on twitter

#below two line are commented becouse data already stored in db no need of it anymore<br />
<p><b>
Woeid = api.trends_available() #<br />
db.WOEID.insert_many(Woeid)
</b></p>

### Selecting Cities in WOEID data 

In [63]:
cities=["Delhi","Kolkata","Bangalore","Surat","India"]
loc = list(db.WOEID.find({"country" : "India",'name':{"$in":cities}},{'_id':0,'name':1,'woeid':1}))


In [64]:
pprint.pprint(loc[1])

{u'name': u'Surat', u'woeid': 2295405}


## Performing Latest Trend Query & Strore them

In [73]:
Trend={}
for place in loc:
    Trend[place["name"]] = api.trends_place(place["woeid"])

### Clean OUr Data drop unneccesary content

In [84]:
for key in Trend:
    for trend in Trend[key][0]['trends']:
        del trend['query']
        del trend["url"]

### Insert Data Into database


In [94]:
for key in Trend:
    db.placesT.insert_one(Trend[key][0])
print "Insertion Complete"


Insertion Complete


In [95]:
#Delete All Unused Variables
del key,Trend

## AnaLysing Data

In [97]:
#extract Data from databse
trend=list(db.placesT.find({}))

In [113]:
#colName=trend[0]['locations'][0]['name']
trendName=[li["name"] for li in  trend[0]['trends']]
tweetVolume=[li["tweet_volume"] for li in trend[0]["trends"]]
tweetVolume

[11525,
 20336,
 19451,
 None,
 14595,
 None,
 None,
 None,
 None,
 11024,
 None,
 14901,
 None,
 None,
 None,
 None,
 74522,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 15682,
 None,
 None,
 None,
 28099,
 42621,
 None,
 None,
 None,
 None,
 None,
 47775,
 88121,
 14279,
 None,
 None,
 None]

In [98]:
trend[0]['as_of']

{u'_id': ObjectId('56b5d2a452b9770fa821a0c7'),
 u'as_of': u'2016-02-05T20:55:17Z',
 u'created_at': u'2016-02-05T20:46:31Z',
 u'locations': [{u'name': u'Kolkata', u'woeid': 2295386}],
 u'trends': [{u'name': u'#Don3ComingSoon',
   u'promoted_content': None,
   u'tweet_volume': 11525},
  {u'name': u'#GujaratLandScam',
   u'promoted_content': None,
   u'tweet_volume': 20336},
  {u'name': u'#earthquake', u'promoted_content': None, u'tweet_volume': 19451},
  {u'name': u'#NFBA2016', u'promoted_content': None, u'tweet_volume': None},
  {u'name': u'#\u0938\u093f\u0902\u0939\u0938\u094d\u0925_\u0915\u0941\u0902\u092d_\u0909\u091c\u094d\u091c\u0948\u0928',
   u'promoted_content': None,
   u'tweet_volume': 14595},
  {u'name': u'Happy Birthday BESA',
   u'promoted_content': None,
   u'tweet_volume': None},
  {u'name': u'Randeep Hooda',
   u'promoted_content': None,
   u'tweet_volume': None},
  {u'name': u'Bagga', u'promoted_content': None, u'tweet_volume': None},
  {u'name': u'Karun Nair', u'promot